# Python - Excel
+ [用Python讀寫Excel文件的方式比較](https://www.itread01.com/content/1535047103.html)
+ XlsxWriter,只能Create 新Excel 不支援讀取與修改
    + [doxls writer doc](https://xlsxwriter.readthedocs.io/index.html)
+ openpyxl (不支援讀取Excel內含公式的儲存格的運算結果！)
    + [當大蟒繩遇上 Excel](https://medium.com/pyradise/%E7%95%B6%E5%A4%A7%E8%9F%92%E8%9B%87%E8%AA%9E%E8%A8%80%E9%81%87%E4%B8%8A%E8%80%81%E7%89%8C%E8%A9%A6%E7%AE%97%E8%A1%A8%E8%BB%9F%E9%AB%94-fc7947792000)
+ xlwings(擁有專屬於Microsoft Excel的擴充套件！,Python 可以當VBA使用！)
    + (xlwin.org)[https://www.xlwings.org/]
+ xlrd/xlwt/(不支援 2010 後)
+ Microsoft Excel API ,致命的缺點：慢到死  
+ [Pandas styling](https://pandas.pydata.org/pandas-docs/stable/user_guide/style.html)

In [76]:
import pandas as pd
import numpy as np
import xlsxwriter 
from datetime import datetime
from xlsxwriter.utility import xl_rowcol_to_cell
pd.__version__

'0.22.0'

In [77]:
now=datetime.now()
FcDay=[pd.date_range('2019-3-1','2019-3-10',freq='1d')]
FcDay2=[pd.date_range('2019-3-1','2019-3-10',freq='2d')]
dus_ts=pd.Series(data=np.random.randn(10),index=FcDay,name='S1')
dus_ts2=pd.Series(data=np.random.randn(5),index=FcDay2,name='S2')

In [78]:
# 日期自動 Mapping
df=pd.concat([dus_ts, dus_ts2], axis=1)

In [79]:
#補 0
df=df.fillna(0)

In [80]:
df['Gap']=df['S1']-df['S2']

In [81]:
# 搭配 EXCEL 公試
df=df.T

In [82]:
df.reset_index(level=0, inplace=True)
df

,index,2019-03-01 00:00:00,2019-03-02 00:00:00,2019-03-03 00:00:00,2019-03-04 00:00:00,2019-03-05 00:00:00,2019-03-06 00:00:00,2019-03-07 00:00:00,2019-03-08 00:00:00,2019-03-09 00:00:00,2019-03-10 00:00:00
0,S1,-0.807564,-1.046053,0.209256,0.209652,0.963964,-0.962992,1.815049,-0.163351,-1.198712,-0.416093
1,S2,1.493636,0.000000,1.125072,0.000000,-0.931850,0.000000,-0.856522,0.000000,-1.796272,0.000000
2,Gap,-2.301200,-1.046053,-0.915816,0.209652,1.895814,-0.962992,2.671571,-0.163351,0.597560,-0.416093


In [145]:
# 土法煉鋼 By Columns寫
workbook = xlsxwriter.Workbook('Excel_01.xlsx')
worksheet = workbook.add_worksheet()
date_format = workbook.add_format({'num_format': 'yy/mm/dd','align': 'left','bg_color':'green'})


# Header
for col_num, value in enumerate(df.columns.values):
    worksheet.write(0, col_num + 1, value[0],date_format)
   # print(0, col_num + 1,value[0])

#RawData , row=df.shape[0]
for i in range(0,df.shape[1]):
    for row, value in enumerate(df.iloc[:,i]):
        #style=cell_F[df.columns[i]]
        #worksheet.write(row+1,i+1, value,style)
        worksheet.write(row+1,i+1, value)


#RawData , row=df.shape[0]
for i in range(df.shape[0],int(df.shape[0])+1):
    for columns, value in enumerate(df.columns.values):
        val="="+xl_rowcol_to_cell(i, columns + 1)+"-"+xl_rowcol_to_cell(i-1, columns + 1)
        worksheet.write(xl_rowcol_to_cell(i+1, columns + 1),val)
        
worksheet.write_array_formula('A1:A1', '{=SUM(B1:C1*B2:C2)}')
workbook.close()